# Spanish Electricity Market Analysis: Part 1/2

The first part of the project is to currently get the data from the specified source and analyse the quality of it. 
In this case, we are retrieving data from Red Eléctrica de España (REE), the Spanish Transmission System Operator, responsible to maintain the grid frequency at 50Hz and to balance consumption and demand sides at every time. 

## Part 1. Dataset import and data cleaning → export to SQL

### 1.1. Libraries import

In [25]:
import pandas as pd
import os
from sqlalchemy import create_engine
import datetime

sns.set()

### 1.2. Reading .csv file for REE Spanish Electricity Market Data

In [26]:
os.getcwd() #we get the current working directory 

'/home/ingrid/Documents/DA_Ironhack/Week5/Project-Week-5-Your-Own-Project/your-project'

In [27]:
raw_data = pd.read_csv('datasets/spain_energy_market.csv') #reading the csv file with the aggregated data from 2014-2018. 

In [28]:
raw_data.head()

,datetime,id,name,geoid,geoname,value
0,2014-01-01 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,25.280833
1,2014-01-02 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,39.924167
2,2014-01-03 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,4.992083
3,2014-01-04 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,4.091667
4,2014-01-05 23:00:00,600,Precio mercado SPOT Diario ESP,3.0,España,13.587500


In [29]:
raw_data.shape #how many rows and columns this df has 

(40212, 6)

In [30]:
raw_data.dtypes #checking the types of the dataframe. 

datetime     object
id            int64
name         object
geoid       float64
geoname      object
value       float64
dtype: object

In [31]:
raw_data.datetime = pd.to_datetime(raw_data['datetime']) #since the column datetime is imported as an string

In [32]:
raw_data.dtypes

datetime    datetime64[ns]
id                   int64
name                object
geoid              float64
geoname             object
value              float64
dtype: object

In [33]:
raw_data.name.unique()

array(['Precio mercado SPOT Diario ESP', 'Precio mercado SPOT Diario FRA',
       'Precio mercado SPOT Diario POR',
       'Energía asignada en Mercado SPOT Diario España',
       'Energía asignada en Mercado SPOT Diario Francia', nan,
       'Rentas de congestión mecanismos implícitos diario Francia exportación',
       'Rentas de congestión mecanismos implícitos diario Portugal exportación',
       'Rentas de congestión mecanismos implícitos diario Francia importación',
       'Rentas de congestión mecanismos implícitos diario Portugal importación',
       'Demanda real', 'Demanda programada PBF total',
       'Generación programada PBF total',
       'Generación programada PBF Eólica',
       'Generación programada PBF Ciclo combinado',
       'Generación programada PBF Carbón',
       'Generación programada PBF Nuclear',
       'Generación programada PBF Gas Natural Cogeneración',
       'Generación programada PBF UGH + no UGH',
       'Generación programada PBF Solar fotovoltaica'

From the names above we can see that the dataset has mixed information. In this table we can found information from different sources: 

- Spot or DAM Market price, for Spain, France and Portugal (Iberian Electricity market system)
- Congestion mechanisms for imbalances based on interconnections with France and Portugal 
- Real electricity demand. 
- Scheduled program generation. (Baseline → Programa Base Funcionamiento)
- Scheduled generation per technology. 

Hence, we should split the entire tables into subtables to be able to perform the data analysis based on our objectives. 

### 1.3. Total scheduled electricity demand (Per day)

In [34]:
PBF_total_df = raw_data[raw_data.name == 'Generación programada PBF total']

In [35]:
PBF_total_df.head()

,datetime,id,name,geoid,geoname,value
23819,2014-01-01 23:00:00,10258,Generación programada PBF total,NaN,NaN,642771.8
23820,2014-01-02 23:00:00,10258,Generación programada PBF total,NaN,NaN,658078.5
23821,2014-01-03 23:00:00,10258,Generación programada PBF total,NaN,NaN,680564.6
23822,2014-01-04 23:00:00,10258,Generación programada PBF total,NaN,NaN,644494.7
23823,2014-01-05 23:00:00,10258,Generación programada PBF total,NaN,NaN,598661.4


From this dataframe we can see that, since we are working with the *total* scheduled electricity for Spain  on that day, there is no need for the geoid, geoname, name and id columns. Also, we will reset the index. 

In [40]:
PBF_total_df = PBF_total_df[['datetime','value']].reset_index(drop=True)
PBF_total_df["date"] = PBF_total_df["datetime"].dt.date

PBF_total_df.drop('datetime', axis=1, inplace=True)
PBF_total_df.set_index('date', inplace=True)


In [41]:
PBF_total_df.head()

,value
date,
2014-01-01,642771.8
2014-01-02,658078.5
2014-01-03,680564.6
2014-01-04,644494.7
2014-01-05,598661.4


With this table we have the total energy demand scheduled for the day after it is calculated. We can export this table to SQL. 

In [18]:
# Workbench Databench 
driver = 'mysql+pymysql'
user = 'root'
password = 'iMc91linux'
ip = 'localhost'
database = 'REE_analysis'


# connection_string to connect to Workbench Database 
connection_string = f'{driver}://{user}:{password}@{ip}/{database}'

    
# Engine creation 
engine = create_engine(connection_string)

In [ ]:

#uploading PBF REE data into mySQL Database 
PBF_total_df.to_sql('PBF_total', engine)

In [126]:
PBF_total_df.to_csv('cleaned_data/PBF_total_df.csv')

### 1.3. importing geolocation tables

In [9]:
geo_id = pd.read_excel('datasets/geolocalizaciones.xlsx')

In [10]:
geo_id.head()

,GEO_ID,NIVEL,NAME
0,1,0,Portugal
1,2,0,Francia
2,3,0,España
3,8739,0,Andorra
4,8740,0,Marruecos


In [42]:
geo_id.set_index('GEO_ID', inplace=True)

In [47]:
geo_id.head()

,NIVEL,NAME
GEO_ID,,
1,0,Portugal
2,0,Francia
3,0,España
8739,0,Andorra
8740,0,Marruecos


As we can see above, the dataset contains a geographic id that explains us which generation unit has been turned on to produce the expected electricity for the time period $t_i$. This table contains three columns that contain the following information: 

- *GEO_ID*: the unique indicator id for each location 
- *NIVEL*: Nivel stands for the level of each location. It is an integer that can go from 0 to 5 and it is referred to: 
    - 0: Country 
    - 1: Areas in Spain (Peninsula, Canary Islands, Melilla, Ceuta, Balearic Area, Melilla)
    - 2: Autonomous communities 
    - 3: Provinces
    - 4: Province capital 
    - 5: Municipalty
    - 6: Hydrological basin
- *NAME*: name of the specific geo_id indicator. Location


In [19]:
geo_id.to_sql('GEO_id', engine)

In [127]:
geo_id.to_csv('cleaned_data/geo_id.csv')

### 1.5. Electricity generation per source-

In [96]:
## variables we want in our dataset
columns= ['Generación programada PBF Eólica', 'Generación programada PBF Ciclo combinado',
       'Generación programada PBF Carbón', 'Generación programada PBF Nuclear', 'Generación programada PBF Gas Natural Cogeneración',
       'Generación programada PBF UGH + no UGH', 'Generación programada PBF Solar fotovoltaica', 'Generación programada PBF Turbinación bombeo']

In [97]:
gen_source_df = raw_data.loc[raw_data['name'].isin(columns)]

In [98]:
gen_source_df.drop(['id', 'geoid','geoname'], axis=1, inplace=True)

/home/linuxbrew/.linuxbrew/opt/python/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [99]:
gen_source_df.head()

,datetime,name,value
25644,2014-01-01 23:00:00,Generación programada PBF Eólica,277443.9
25645,2014-01-02 23:00:00,Generación programada PBF Eólica,237323.1
25646,2014-01-03 23:00:00,Generación programada PBF Eólica,329506.6
25647,2014-01-04 23:00:00,Generación programada PBF Eólica,292138.3
25648,2014-01-05 23:00:00,Generación programada PBF Eólica,223942.0


In [100]:
gen_source_df["date"] = gen_source_df["datetime"].dt.date

/home/linuxbrew/.linuxbrew/opt/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [101]:
gen_source_df = gen_source_df.drop('datetime', axis=1)


In [102]:
gen_source_df = gen_source_df[['date', 'name', 'value']]

In [103]:
gen_source_df.head()

,date,name,value
25644,2014-01-01,Generación programada PBF Eólica,277443.9
25645,2014-01-02,Generación programada PBF Eólica,237323.1
25646,2014-01-03,Generación programada PBF Eólica,329506.6
25647,2014-01-04,Generación programada PBF Eólica,292138.3
25648,2014-01-05,Generación programada PBF Eólica,223942.0


In [110]:
#we rename the name column according to the source, in order to facilitate the visualization and understanding. 

sources = {'Generación programada PBF Eólica':'wind_power', 'Generación programada PBF Ciclo combinado':'CHP',
           'Generación programada PBF Carbón':'coal', 'Generación programada PBF Nuclear':'nuclear', 
           'Generación programada PBF Gas Natural Cogeneración':'natural_gas',
           'Generación programada PBF UGH + no UGH':'hydro_1', 
           'Generación programada PBF Solar fotovoltaica':'solar_PV', 
           'Generación programada PBF Turbinación bombeo':'hydro_2'}

gen_source_df = gen_source_df.replace(sources) 

In [113]:
#change the name of the column 'name' for source

gen_source_df = gen_source_df.rename(columns={'name':'source', 'value':'value_MWh'})

In [114]:
gen_source_df.head()

,date,source,value_MWh
25644,2014-01-01,wind_power,277443.9
25645,2014-01-02,wind_power,237323.1
25646,2014-01-03,wind_power,329506.6
25647,2014-01-04,wind_power,292138.3
25648,2014-01-05,wind_power,223942.0


In [128]:
gen_source_df.to_csv('cleaned_data/generation_per_source.csv')